In [ ]:
# agent_router_with_memory.py

import os
import re
import numpy as np
from typing import Any, Dict, List, TypedDict

from langchain import LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseMemory
from langchain.memory import ConversationBufferMemory, CombinedMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import ChatOpenAI  # only if you still want OpenAI elsewhere
from langchain.agents import Tool, AgentType, create_pandas_dataframe_agent
from langchain.embeddings.openai import OpenAIEmbeddings

from deep_translator import GoogleTranslator
from sklearn.metrics.pairwise import cosine_similarity
from langgraph.graph import StateGraph, START, END

# ——— 1) Replace LlamaCpp with a HuggingFace pipeline ——————————————

from transformers import pipeline, LlamaForCausalLM, LlamaTokenizer

# 1a) point this at your local HF directory (or model name on HF Hub)
MODEL_PATH = "/path/to/your/llama-3.2-3b-checkpoint/"

tokenizer = LlamaTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
model     = LlamaForCausalLM.from_pretrained(MODEL_PATH, device_map="auto", torch_dtype="auto")

# build a standard text-generation pipeline
text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.2,
    top_p=0.9,
    device_map="auto",
)

# wrap it for LangChain
hf_llm = HuggingFacePipeline(pipeline=text_gen)


# ——— 2) Tools you still want to keep ——————————————————————————————

def calculator_tool(expr: str) -> str:
    try:
        return f"The result of '{expr}' is {eval(expr)}"
    except Exception as e:
        return f"Error: {e}"
calculator = Tool("Calculator", calculator_tool, "Basic arithmetic")

def sentiment_analysis_tool(text: str) -> str:
    t = text.lower()
    if any(w in t for w in ("happy","great","good","excellent")):
        return "Positive 👍"
    if any(w in t for w in ("sad","bad","terrible","awful")):
        return "Negative 👎"
    return "Neutral 🤔"
sentiment = Tool("SentimentAnalysis", sentiment_analysis_tool, "Sentiment detection")


# ——— 3) Build your Pandas-DataFrame agent with the same HF LLM —————————

# assuming `df` is already defined earlier in your script
pandas_agent = create_pandas_dataframe_agent(
    hf_llm,         # use HuggingFacePipeline instead of ChatOpenAI
    df,
    verbose=True,
    allow_dangerous_code=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,  # or your preferred type
)


# ——— 4) Memory & caching scaffolding —————————————————————————————

_long_term: Dict[str,List[str]] = {}
_qa_cache:    Dict[str,str]   = {}
_qa_embeds:   Dict[str,List[float]] = {}
SESSION_ID = "user_123"

def update_long_term_memory(user: str, inp: str, outp: str):
    mem = _long_term.setdefault(user, [])
    if inp:  mem.append(f"User: {inp}")
    if outp: mem.append(f"Bot:  {outp}")
    _long_term[user] = mem[-10:]

def get_long_term_memory(user: str) -> str:
    return "\n".join(_long_term.get(user, []))

EMB = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
def find_similar_cached(q: str, threshold: float=0.85) -> str:
    vec = EMB.embed_query(q)
    for orig,ov in _qa_embeds.items():
        if cosine_similarity([vec],[ov])[0][0] >= threshold:
            return orig
    return None

TOOL_KEYWORDS = ["calculate","translate","search","find","what","give me","list","summarize","analyze"]
def is_tool_query(txt: str) -> bool:
    low = txt.lower().strip()
    if ";" in txt or "\n" in txt: return True
    return any(low.startswith(kw) for kw in TOOL_KEYWORDS)


# ——— 5) Router via LangGraph (unchanged except we only keep 3 tools) ————

class RouterState(TypedDict):
    input: str
    decision: str
    output: Any

def llm_call_router(state: RouterState) -> RouterState:
    t = state["input"].strip().lower()
    if t.startswith("calculate"):    state["decision"] = "Calculator"
    elif "sentiment" in t or any(w in t for w in ("happy","sad")):
        state["decision"] = "SentimentAnalysis"
    else:
        state["decision"] = "PandasDataAnalysis"
    return state

def run_Calculator(st):    st["output"]=calculator_tool(st["input"]); return st
def run_Sentiment(st):     st["output"]=sentiment_analysis_tool(st["input"]); return st
def run_Pandas(st):        st["output"]=pandas_agent.run(st["input"]); return st

router = StateGraph(state_schema=RouterState)
router.add_node("router", llm_call_router)
router.add_node("Calculator", run_Calculator)
router.add_node("SentimentAnalysis", run_Sentiment)
router.add_node("PandasDataAnalysis", run_Pandas)
router.add_edge(START, "router")
router.add_conditional_edges("router", lambda s: s["decision"],
                            {"Calculator":"Calculator",
                             "SentimentAnalysis":"SentimentAnalysis",
                             "PandasDataAnalysis":"PandasDataAnalysis"})
for node in ("Calculator","SentimentAnalysis","PandasDataAnalysis"):
    router.add_edge(node, END)
router.set_entry_point("router")
router_workflow = router.compile()


# ——— 6) Pure-chat chain, now on HF Llama 3.2 ‐3B ———————————————

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI assistant."),
    ("system","Chat history:\n{chat_history}"),
    ("system","Long-term memory:\n{long_term_memory}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{input}")
])

short_term = ConversationBufferMemory(memory_key="chat_history", input_key="input", return_messages=True)
long_term  = CombinedMemory  # placeholder not used here
combined  = CombinedMemory(memories=[short_term,
                LongTermChatMemory(session_id=SESSION_ID)])

chat_chain = LLMChain(
    prompt=prompt,
    llm=hf_llm,   # our HuggingFacePipeline
    memory=combined
)


# ——— 7) unify in `converse()` ——————————————————————————————————

def converse(user_input: str) -> str:
    low = user_input.lower().strip()
    if "remember" in low:
        return "Memory:\n"+get_long_term_memory(SESSION_ID)

    if is_tool_query(user_input):
        sim = find_similar_cached(user_input)
        if sim:
            return _qa_cache[sim]
        st  = router_workflow.invoke({"input": user_input})
        out = st["output"]
        _qa_cache[user_input]  = out
        _qa_embeds[user_input] = EMB.embed_query(user_input)
        update_long_term_memory(SESSION_ID, user_input, out)
        return out

    out = chat_chain.run(user_input)
    update_long_term_memory(SESSION_ID, user_input, out)
    return out


# ——— 8) quick smoke‐test ———————————————————————————————————

if __name__ == "__main__":
    for q in [
        "Calculate 3 * 8",
        "How am I feeling? Sentiment 'I am sad'",
        "Show me summary stats on the DataFrame",
        "Do you remember our previous requests?"
    ]:
        print("→", converse(q))
